In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import warnings
import os

# Scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# XGBoost
import xgboost as xgb

# Cấu hình hiển thị
pd.set_option('display.float_format', lambda x: '%.4f' % x)
warnings.filterwarnings('ignore')

# Kiểm tra GPU
import tensorflow as tf
print("🔥 GPU Status:", tf.config.list_physical_devices('GPU'))
print("✅ Sẵn sàng chiến đấu với 73k dòng dữ liệu!")

🔥 GPU Status: []
✅ Sẵn sàng chiến đấu với 73k dòng dữ liệu!


In [2]:
# Đường dẫn file mới (73k dòng)
DATA_PATH = '../data/processed/clean_vn_housing.csv'

df = pd.read_csv(DATA_PATH)
print(f"📂 Đã load dữ liệu: {df.shape}")
# Kỳ vọng: (73652, 358)

# 1. Tách Feature & Target
target_col = 'Total_Price_Billion'

# Loại bỏ các cột không dùng để train
# Lưu ý: File sạch đã bỏ Address và Price_per_m2, nhưng ta cứ drop lần nữa cho chắc
drop_cols = [target_col, 'Price_per_m2', 'Address', 'Unnamed: 0']
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# 2. Log Transform Target
# BẮT BUỘC: Giúp model dự đoán tốt dải giá rộng từ 1 tỷ đến 100 tỷ
y = np.log1p(df[target_col])

# 3. Chia tập Train/Test (80/20)
# Với 73k dòng, tập test sẽ có khoảng 14.7k dòng -> Rất đủ để đánh giá tin cậy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Scaling (RobustScaler)
# Cực quan trọng khi có cột Length/Width/Area biên độ chênh lệch lớn
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lưu Scaler cho App
if not os.path.exists('../models'): os.makedirs('../models')
joblib.dump(scaler, '../models/scaler_final.pkl')

print(f"✅ Dữ liệu đã chuẩn hóa. Số Features: {X_train.shape[1]}")

📂 Đã load dữ liệu: (73652, 358)
✅ Dữ liệu đã chuẩn hóa. Số Features: 357


In [3]:
def danh_gia_hieu_nang(model, X_test, y_test_log, ten_mo_hinh):
    # Dự đoán
    y_pred_log = model.predict(X_test)

    # Chuyển ngược Log -> Giá thực (Tỷ VNĐ)
    y_true = np.expm1(y_test_log)
    y_pred = np.expm1(y_pred_log)
    y_pred = np.maximum(y_pred, 0) # Chặn số âm

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"\n{'='*40}")
    print(f"📊 KẾT QUẢ: {ten_mo_hinh.upper()}")
    print(f"{'='*40}")
    print(f"MAE  : {mae:.4f} Tỷ")
    print(f"RMSE : {rmse:.4f} Tỷ")
    print(f"R2   : {r2:.4f} ({r2*100:.2f}%)")

    if r2 > 0.7: print("   🚀 Model đã đạt mức độ chính xác cao!")
    elif r2 > 0.8: print("   🏆 Model xuất sắc (SOTA)!")

    return {'Model': ten_mo_hinh, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

In [4]:
print(">>> 1. Tuning Random Forest (Big Data Optimized)...")
t0 = time.time()

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid_rf = {
    # Giữ số cây vừa phải để không bị treo máy
    'n_estimators': [200, 300],

    # Cho phép cây sâu hơn để học các quy luật phức tạp của Tên đường/Mặt tiền
    'max_depth': [20, 30],

    # QUAN TRỌNG: 0.4 nghĩa là mỗi lần tách nhánh xem xét 40% số cột (khoảng 140 cột)
    # Giúp cây "nhìn" thấy Tên đường và Kích thước rõ hơn
    'max_features': [0.4, 'sqrt'],

    # Tăng min_samples_leaf lên 4 để tránh học vẹt các căn nhà cá biệt
    'min_samples_leaf': [2, 4]
}

# Dùng n_jobs=-1 để ép CPU chạy 100% công suất
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_rf.fit(X_train_scaled, y_train)

best_rf = grid_rf.best_estimator_
print(f"⏱️ RF Time: {time.time() - t0:.1f}s")
print("Best Params RF:", grid_rf.best_params_)

# Lưu và đánh giá
joblib.dump(best_rf, '../models/rf_final.pkl')
res_rf = danh_gia_hieu_nang(best_rf, X_test_scaled, y_test, "Random Forest (Full Data)")

>>> 1. Tuning Random Forest (Big Data Optimized)...
Fitting 3 folds for each of 16 candidates, totalling 48 fits
⏱️ RF Time: 819.8s
Best Params RF: {'max_depth': 30, 'max_features': 0.4, 'min_samples_leaf': 2, 'n_estimators': 300}

📊 KẾT QUẢ: RANDOM FOREST (FULL DATA)
MAE  : 0.7799 Tỷ
RMSE : 1.3050 Tỷ
R2   : 0.7811 (78.11%)
   🚀 Model đã đạt mức độ chính xác cao!


In [5]:
print(">>> 2. Tuning XGBoost (GPU RTX 3050 Power)...")
t0 = time.time()

# Cấu hình chuẩn cho GPU
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist', # Bắt buộc cho dữ liệu lớn (>50k dòng)
    device='cuda',      # Kích hoạt GPU
    random_state=42,
    n_jobs=1
)

param_grid_xgb = {
    # Tăng số cây lên 3000 vì dữ liệu giờ đã rất lớn, cần nhiều cây để học hết
    'n_estimators': [2000, 3000],

    # Học chậm (0.01) để chắt chiu từng % độ chính xác
    'learning_rate': [0.01, 0.02],

    # Độ sâu: Giữ 8-10 để an toàn VRAM
    'max_depth': [8, 10],

    # QUAN TRỌNG: colsample_bytree = 0.5
    # Giúp model chọn lọc trong 358 cột, không bị nhiễu bởi các tên đường lạ
    'colsample_bytree': [0.5, 0.6],

    # Lấy mẫu dòng: 0.8
    'subsample': [0.8],

    # Regularization: Tăng nhẹ để chống lại sự phân mảnh của dữ liệu One-Hot
    'reg_alpha': [2, 5],
    'reg_lambda': [5, 10]
}

grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='r2', verbose=1, n_jobs=1)
grid_xgb.fit(X_train_scaled, y_train)

best_xgb = grid_xgb.best_estimator_
print(f"⏱️ XGB Time: {time.time() - t0:.1f}s")
print("Best Params XGB:", grid_xgb.best_params_)

# Lưu model (Model này sẽ dùng cho App)
res_xgb = danh_gia_hieu_nang(best_xgb, X_test_scaled, y_test, "XGBoost (Full Data)")
joblib.dump(best_xgb, '../models/xgb_final.pkl')

>>> 2. Tuning XGBoost (GPU RTX 3050 Power)...
Fitting 3 folds for each of 64 candidates, totalling 192 fits
⏱️ XGB Time: 4865.8s
Best Params XGB: {'colsample_bytree': 0.6, 'learning_rate': 0.02, 'max_depth': 10, 'n_estimators': 3000, 'reg_alpha': 2, 'reg_lambda': 5, 'subsample': 0.8}

📊 KẾT QUẢ: XGBOOST (FULL DATA)
MAE  : 0.7739 Tỷ
RMSE : 1.3105 Tỷ
R2   : 0.7792 (77.92%)
   🚀 Model đã đạt mức độ chính xác cao!


['../models/xgb_final.pkl']

In [6]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
print(">>> 3. Training Stacking Ensemble (Kết hợp sức mạnh)...")
t0 = time.time()

# Định nghĩa các model con (Base Learners) bằng bộ tham số tốt nhất vừa tìm được
estimators = [
    ('rf', best_rf),
    ('xgb', best_xgb)
]

# Mô hình trùm (Final Estimator): Dùng Ridge Regression để tổng hợp
# Ridge giúp cân bằng trọng số giữa RF và XGB, tránh phụ thuộc quá vào 1 model
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=1.0),
    n_jobs=-1,
    cv=3 # Cross-validation nội bộ để train model trùm
)

stacking_model.fit(X_train_scaled, y_train)

print(f"⏱️ Stacking Time: {time.time() - t0:.1f}s")
res_stack = danh_gia_hieu_nang(stacking_model, X_test_scaled, y_test, "Stacking Ensemble")

>>> 3. Training Stacking Ensemble (Kết hợp sức mạnh)...
⏱️ Stacking Time: 197.5s

📊 KẾT QUẢ: STACKING ENSEMBLE
MAE  : 0.7669 Tỷ
RMSE : 1.2850 Tỷ
R2   : 0.7877 (78.77%)
   🚀 Model đã đạt mức độ chính xác cao!


In [7]:
from sklearn.ensemble import VotingRegressor

print(">>> 3. Training Voting Regressor (Kết hợp sức mạnh)...")
# Lấy 2 model tốt nhất vừa train xong
estimators = [
    ('rf', best_rf),
    ('xgb', best_xgb)
]

# Voting: Lấy trung bình cộng kết quả dự đoán của cả 2
voting_model = VotingRegressor(estimators=estimators, n_jobs=-1)
voting_model.fit(X_train_scaled, y_train)

# Đánh giá
joblib.dump(voting_model, '../models/voting_final.pkl')
res_voting = danh_gia_hieu_nang(voting_model, X_test_scaled, y_test, "Voting Ensemble (Final Boss)")

>>> 3. Training Voting Regressor (Kết hợp sức mạnh)...

📊 KẾT QUẢ: VOTING ENSEMBLE (FINAL BOSS)
MAE  : 0.7676 Tỷ
RMSE : 1.2833 Tỷ
R2   : 0.7883 (78.83%)
   🚀 Model đã đạt mức độ chính xác cao!
